In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import hydra
import os
import pathlib
import pandas as pd

In [ ]:
from smc01.postprocessing.dataset import SMCParquetDataset
from smc01.postprocessing.cli.util import make_dataloader

In [ ]:
DATA_DIR = pathlib.Path(os.getenv('DATA_DIR'))
DATASET_DIR = DATA_DIR / 'interpolated/2022-04-20-full/'

In [ ]:
with hydra.initialize_config_module('smc01.postprocessing.conf'):
    cfg = hydra.compose('train', ['experiment=attention_gdps_metar', 'experiment.dataset.shuffle_buffer_size=10'])

In [ ]:
d = hydra.utils.instantiate(cfg.experiment.dataset)

In [ ]:
d

In [ ]:
loader = make_dataloader(cfg, d)

In [ ]:
sample = next(iter(loader))

In [ ]:
sample['station_id']

In [ ]:
d.buffer_size

In [ ]:
for i in range(10):
    sample = next(iter(d))
    print(sample['forecast_id'][0], sample['step_id'][0])

In [ ]:
for i in range(10):
    sample = next(iter(d))
    print(sample['forecast_id'][0], sample['step_id'][0])

In [ ]:
dir(iter(d))

In [ ]:
next(iter(d))

In [ ]:
loader = make_dataloader(
        cfg, d, concat_collate=cfg.experiment.concat_collate
    )

In [ ]:
sample = next(iter(loader))

In [ ]:
sample.keys()

In [ ]:
cfg.experiment.concat_collate

In [ ]:
sample['forecast'].shape

In [ ]:
sample['forecast_id']

In [ ]:
sample['forecast_id'][:,0].unsqueeze(0).shape

In [ ]:
((sample['model_id'] == sample['model_id'][:,0].unsqueeze(1)) | sample['padding']).all()

In [ ]:
sample['step_id'].shape

In [ ]:
sample['features'].shape

In [ ]:
d = SMCParquetDataset(DATASET_DIR)

In [ ]:
sample = d[-1]

In [ ]:
sample

In [ ]:
major_version = pd.Series(index=sample.index, data=0)

In [ ]:
major_version.loc[sample['forecast_time'] < '2019-07-03'] = 1

In [ ]:
major_version.loc[(sample['forecast_time'] >= '2019-07-03') & (sample['forecast_time'] < '2021-12-01')] = 2

In [ ]:
major_version.loc[sample['forecast_time'] > '2021-12-01'] = 3

In [ ]:
major_version